In [1]:
# sparse retriever test를 위한 doc_list 생성
from loader.load_documents import *
# 1. 경로 설정
timetable_path = "data/Timetable_Crawling_Data"
guidebook_path = "data/major_guide_2025.pdf"
pram_path = "data/pram_2025_1.json"
prof_path = "data/hufs_professor.json"
college_path = "data/hufs_colleges.json"
notice_path = "data/hufs_notice.json"
schedule_path = "data/hufs_schedule.json"

# 2. 문서 로드
print("강의시간표 로드 중...")
subject_docs = load_all_subject_documents(timetable_path)

print("전공가이드북 로드 중...")
guidebook_docs = load_major_guidebook(guidebook_path)

print("수강편람 로드 중...")
pram_docs = load_sugang_pram(pram_path)

print("교수진 정보 로드 중...")
prof_docs = load_professor_documents(prof_path)

print("단과대 정보 로드 중...")
college_docs = load_college_intro(college_path)

print("학사공지 로드 중...")
notice_docs = load_notice_documents(notice_path)

print("학사일정 로드 중...")
schedule_docs = load_academic_schedule(schedule_path)

doc_list = subject_docs + guidebook_docs + pram_docs + prof_docs + college_docs + notice_docs +schedule_docs

강의시간표 로드 중...
전공가이드북 로드 중...
수강편람 로드 중...
교수진 정보 로드 중...
단과대 정보 로드 중...
학사공지 로드 중...
학사일정 로드 중...


In [2]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# doc_list가 Document 객체 리스트일 경우, text만 추출
texts = [doc.page_content for doc in doc_list]

# bm25 retriever 초기화
bm25_retriever = BM25Retriever.from_texts(texts)
bm25_retriever.k = 20  # BM25Retriever의 검색 결과 개수

# 실험에 따라 변경될 내용
pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, pinecone_retriever],
    weights=[0.3, 0.7]
)

NameError: name 'vectorstore' is not defined

In [4]:
import os, time
from dotenv import load_dotenv
from uuid import uuid4
from operator import itemgetter

from langchain.vectorstores import Pinecone as PineconeVectorStore
from pinecone import Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.document_transformers import LongContextReorder
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.chains import create_retrieval_chain

# .env 파일에서 API 키 로드
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 축약어 대응용
def normalize_query(query: str) -> str:
    abbreviation_map = {
        "글스산": "글로벌스포츠산업전공",
        "바메공": "바이오메디컬공학전공",
        "BME": "바이오메디컬공학전공",
        "이중": "이중전공",
        "통대": "통번역대학",
        "공대": "공과대학",
        "일통": "일본어통번역학과",
        "영통": "영어통번역학과",
        "독통": "독일어통번역학과",
        "스통": "스페인어통번역학과",
        "마인어":"말레이시아인도네시아통번역학과",
        "GBT": "Global Business&Technology",
        "융인": "융합인재학과",
        "중통": "중국어통번역학과",
        "국금": "국제금융학과",
        "이통": "이탈리아어통번역학과",
        "태통": "태국어통번역학과",
        "정통": "정보통신공학과",
        "산공": "산업경영공학과",
        "산경공": "산업경영공학과",
        "파에": "Finance&AI융합학부",
        "데융": "AI데이터융합학부",
        "글자전": "글로벌자유전공학부",
        "자전": "글로벌자유전공학부",
        "대영": "대학영어",
        "데사":"데이터사이언스",
        "국리":"국가리더전공",
        "세크": "세르비아·크로아티아",
        "그불": "그리스·불가리아",
        "전물": "전자물리학과",
        "생공": "생명공학과",
        "디콘": "디지털콘텐츠학부",
        "자대": "자연과학대학",
        "인경관": "인문경상관",
        "국지대": "국제지역대학",
        "전언대": "국가전략언어대학",
        "전언": "국가전략언어",
        "국전언": "국가전략언어대학",
        "융소": "융복합소프트웨어전공"
        # 필요시 추가
    }
    for short, full in abbreviation_map.items():
        query = query.replace(short, full)
    return query

# Pinecone 설정 (불러오기 전용)
def initialize_pinecone():
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "hufs-chatbot"  

    index = pc.Index(index_name)

    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002",
        api_key=OPENAI_API_KEY
    )

    vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="page_content")
    return vectorstore

# OpenAI LLM 로드
def load_model():
    model = ChatOpenAI(
        temperature=0,
        model_name="gpt-4o-mini",  # 테스트용, 향 후 gpt-4o-mini로 변경경
        api_key=OPENAI_API_KEY,
        streaming=True
    )
    print("model loaded...")
    return model

# RAG 체인 구성
def rag_chain(vectorstore):
    llm = load_model()

    # 고급 리트리버 구성
    reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor_15 = CrossEncoderReranker(model=reranker_model, top_n=15)
    vs_retriever30 = vectorstore.as_retriever(search_kwargs={"k": 30})
    retriever = ContextualCompressionRetriever(base_compressor=compressor_15, base_retriever=vs_retriever30)

    # 히스토리 인식 리트리버 구성
    system_prompt = (
        "주어진 대화 기록과 최근 질문을 바탕으로, 이전 대화 내용을 참고하여 "
        "독립적으로 이해 가능한 질문으로 재구성하거나 그대로 사용하세요."
    )
    contextualize_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])
    history_aware_retriever_modified = create_history_aware_retriever(
        llm,
        retriever,
        contextualize_prompt
    )

    # 문서 재정렬 (긴 문서 대응)
    reordering = LongContextReorder()

    my_retriever = (
        {"input": itemgetter("input"),
         "chat_history": itemgetter("chat_history")} |
        history_aware_retriever_modified |
        RunnableLambda(lambda docs: reordering.transform_documents(docs))
    )

    # QA 프롬프트
    '''
    qa_system_prompt = """당신은 한국외국어대학교 학사 행정 정보를 제공하는 챗봇입니다.
    아래 문서를 참고하여 정확하고 간결하게 질문에 답변하세요. 단 일부 질의에 대해서는 문서를 종합적으로 고려하여 답변하는 것이 중요합니다.
    만약 문서에 정보가 없다면 모른다고 답하세요. 반드시 한국어로 높임말을 사용하여 답변하세요.
    
    {context}
    """
    '''
    
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])

    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    return create_retrieval_chain(my_retriever, question_answer_chain)

# 세션 기록 저장소
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 대화 흐름 기반 RAG 체인 초기화
def initialize_conversation(vectorstore):
    base_rag_chain = rag_chain(vectorstore)

    return RunnableWithMessageHistory(
        base_rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )

In [12]:
# Pinecone 벡터스토어 불러오기
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeClient
import os, time
from dotenv import load_dotenv
from uuid import uuid4
from operator import itemgetter

from pinecone import Pinecone
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.document_transformers import LongContextReorder
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.chains import create_retrieval_chain

# .env 파일에서 API 키 로드
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def normalize_query(query: str) -> str:
    abbreviation_map = {
        "글스산": "글로벌스포츠산업전공",
        "바메공": "바이오메디컬공학전공",
        "BME": "바이오메디컬공학전공",
        "이중": "이중전공",
        "통대": "통번역대학",
        "공대": "공과대학",
        "일통": "일본어통번역학과",
        "영통": "영어통번역학과",
        "독통": "독일어통번역학과",
        "스통": "스페인어통번역학과",
        "마인어":"말레이시아인도네시아통번역학과",
        "GBT": "Global Business&Technology",
        "융인": "융합인재학과",
        "중통": "중국어통번역학과",
        "국금": "국제금융학과",
        "이통": "이탈리아어통번역학과",
        "태통": "태국어통번역학과",
        "정통": "정보통신공학과",
        "산공": "산업경영공학과",
        "산경공": "산업경영공학과",
        "파에": "Finance&AI융합학부",
        "데융": "AI데이터융합학부",
        "글자전": "글로벌자유전공학부",
        "자전": "글로벌자유전공학부",
        "대영": "대학영어",
        "데사":"데이터사이언스",
        "국리":"국가리더전공",
        "세크": "세르비아·크로아티아",
        "그불": "그리스·불가리아",
        "전물": "전자물리학과",
        "생공": "생명공학과",
        "디콘": "디지털콘텐츠학부",
        "자대": "자연과학대학",
        "인경관": "인문경상관",
        "국지대": "국제지역대학",
        "전언대": "국가전략언어대학",
        "전언": "국가전략언어",
        "국전언": "국가전략언어대학",
        "융소": "융복합소프트웨어전공"
        # 필요시 추가
    }
    
    for short, full in abbreviation_map.items():
        query = query.replace(short, full)
    return query

#from langchain.vectorstores import Pinecone as PineconeVectorStore
#from langchain.embeddings import OpenAIEmbeddings

def initialize_pinecone() -> PineconeVectorStore:
    embedding = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)
    
    vectorstore = PineconeVectorStore.from_existing_index(
        index_name="hufs-chatbot",
        embedding=embedding,
        text_key="page_content"
    )
    return vectorstore

# OpenAI LLM 모델 로드
def load_model():
    return ChatOpenAI(temperature=0, model_name="gpt-4o-mini", api_key=OPENAI_API_KEY, streaming=True)

# RAG 체인 구성 함수 (BM25 + Pinecone 앙상블)
def rag_chain_with_ensemble(doc_list, vectorstore):
    llm = load_model()

    texts = [doc.page_content for doc in doc_list]
    bm25_retriever = BM25Retriever.from_texts(texts)
    bm25_retriever.k = 20

    pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
    ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, pinecone_retriever], weights=[0.3, 0.7])

    reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
    reranker = CrossEncoderReranker(model=reranker_model, top_n=10)
    compression_retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=ensemble_retriever)

    system_prompt = (
        "주어진 대화 기록과 최근 질문을 바탕으로, 이전 대화 내용을 참고하여 "
        "독립적으로 이해 가능한 질문으로 재구성하거나 그대로 사용하세요."
    )

    contextual_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])

    history_aware = create_history_aware_retriever(llm, compression_retriever, contextual_prompt)
    reorder = LongContextReorder()

    my_retriever = ({"input": itemgetter("input"), "chat_history": itemgetter("chat_history")} |
                     history_aware |
                     RunnableLambda(lambda docs: reorder.transform_documents(docs)))

    qa_system_prompt = """
        너는 한국외국어대학교 학생들의 학사 관련 질문에 답변하는 AI 챗봇이야.
        
        - 사용자는 일상적인 표현, 축약어, 은어 등을 쓸 수 있어.
        - 항상 문서에서 제공한 정보에 기반하여 '정확하고 공손하게' 답변해.
        - 질문자의 핵심 의도에 집중해, 관련 없는 부가정보는 생략해.
        - 질문을 받으면 먼저 '의미를 해석하고', 관련 문서에서 '어떤 근거로 어떤 판단을 할 수 있는지' 논리적으로 연결해서 답변해.
        - 문서에 정보가 부족하면 "담당 교수님께 문의하라"는 안내로 마무리해.
        - 항상 끝에 "혹시 더 도와드릴까요?" 와 비슷한 '부드러운 후속 안내'를 덧붙여.
        - 질문에 포함된 표현이 한국어일 경우, 동일한 의미의 영어 표현도 함께 고려해해  
        예: “딥러닝” → “Deep Learning”, “데베” → “데이터베이스” →  “Data Base ”
        가능하면 질문에서 추출된 의미를 한국어/영어 키워드 모두로 확장해서 관련 문서를 찾아봐봐.
        
        ---
        
        [축약어 해석 규칙]
        - "일통" → "일본어통번역학과"
        - "글스산" → "글로벌스포츠산업학부"
        - "전필" → "전공필수", "전선" → "전공선택"
        - "유고" → "유고결석", "공결" → "공식결석"
        - "[언어명]통" → "[언어명]어통번역학과"로 일반화 가능
                                            
        ---
        
        📘 [예시 응답 패턴]
        
        Q: 감기로 병원 다녀왔는데 진료확인서로 유고결석 가능해?  
        💭 (생각) ‘감기’는 병결 사유이고, '진료확인서'는 증빙 서류임. 유고결석은 병원급 이상 서류를 요구함.  
        💬 감기로 병원에 다녀오신 경우, 일반 의원에서 발급한 진료확인서는 인정되지 않을 수 있습니다.  
        병원급 이상의 의료기관에서 받은 진료확인서를 제출하시면 유고결석 처리가 가능할 수 있습니다.  
        정확한 기준은 과목 담당 교수님께 확인해 주세요. 혹시 더 도와드릴까요?
        
        Q: 일통 전필 뭐야?  
        💭 (생각) '일통'은 '일본어통번역학과', '전필'은 전공필수 과목을 의미함.  
        💬 일본어통번역학과의 전공필수 과목은 다음과 같습니다...
        
        Q: 취업계 내면 결석 인정되나요?  
        💭 (생각) 취업계는 공결 사유에 해당되기도 하지만, 교수 재량에 따라 달라질 수 있음.  
        💬 일부 과목에서는 취업계 제출 시 공결로 인정되기도 합니다.  
        다만 교수님에 따라 판단 기준이 다르니 꼭 수업 담당 교수님께 먼저 문의하시기 바랍니다. 혹시 더 도와드릴까요?
        
        ---
        
        📥 사용자 질문:  
        {input}
        
        📄 참고 문서:  
        {context}
        
        💭 (질문 해석 및 내부 판단):  
        (질문의 의미를 해석하고 관련 정보를 연결하는 사고 과정을 간단히 서술)
        
        💬 (최종 응답):  
        (문서 기반으로 논리적인 흐름을 따라 공손하고 명확하게 응답)
        """
    
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ])
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    return create_retrieval_chain(my_retriever, qa_chain)

# 세션 기반 체인 래핑
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

store = {}

def initialize_conversation(doc_list, vectorstore):
    chain = rag_chain_with_ensemble(doc_list, vectorstore)
    return RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )


In [13]:
# 벡터스토어 로딩
vectorstore = initialize_pinecone()

from loader.load_documents import *

# 문서 로드
subject_docs = load_all_subject_documents("data/Timetable_Crawling_Data")
guidebook_docs = load_major_guidebook("data/major_guide_2025.pdf")
pram_docs = load_sugang_pram("data/pram_2025_1.json")
prof_docs = load_professor_documents("data/hufs_professor.json")
college_docs = load_college_intro("data/hufs_colleges.json")
notice_docs = load_notice_documents("data/hufs_notice.json")
schedule_docs = load_academic_schedule("data/hufs_schedule.json")

doc_list = subject_docs + guidebook_docs + pram_docs + prof_docs + college_docs + notice_docs + schedule_docs


In [14]:
# 유니크 세션 ID 생성
session_id = str(uuid4())

# RAG 체인 불러오기
chain = initialize_conversation(doc_list, vectorstore)


C:\Users\samsung-user\anaconda3\envs\boo_chat\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# 세션 ID 지정
session_id = "test_session_01"
query = '저번 학기 국제스포츠산업론 교수님 누구셨지? 연락처 필요한데'
response = chain.invoke({
    "input": query,
}, config={"configurable": {"session_id": session_id}})

print(response)

{'input': '저번 학기 국제스포츠산업론 교수님 누구셨지? 연락처 필요한데', 'chat_history': [], 'context': [Document(id='2f0ce693-2b3f-4ca1-9b0b-7e098d24adc3', metadata={'doc_type': '교수진 정보', '연구분야': 'esports, sport media, sport economics', '연구실': '어문학관 423호', '이름': '류윤지 교수', '이메일': 'yoonji.ryu@hufs.ac.kr', '전화번호': '031-330-4221', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': 'Ph. D Seoul National University, Sport Management'}, page_content='류윤지 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Ph. D Seoul National University, Sport Management 학위를 가지고 있습니다. 연구분야는 esports, sport media, sport economics입니다. 이메일은 yoonji.ryu@hufs.ac.kr입니다. 연구실은 어문학관 423호입니다.'), Document(id='03cf0571-8c54-4661-8187-9dc98f323d39', metadata={'doc_type': '교수진 정보', '연구분야': '스포츠법', '연구실': '정보없음', '이름': '박지훈 교수', '이메일': 'powerpitcher61@gmail.com', '전화번호': '정보없음', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': 'Seoul National University, English literature'}, page_content='박지훈 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Seoul National University, English literatu

In [ ]:
# 세션 ID 지정
#session_id = "test_session_01"
query = '저번 학기 교수님 누구셨지? 연락처 필요한데'
response = chain.invoke({
    "input": query,
}, config={"configurable": {"session_id": session_id}})

print(response)

In [16]:
print(response["answer"])

💭 (질문 해석) '국제스포츠산업론' 과목의 교수님을 찾고 있으며, 그 교수님의 연락처도 필요하다는 의미입니다.

💬 저번 학기 '국제스포츠산업론' 과목의 교수님은 류윤지 교수님입니다. 이메일 주소는 yoonji.ryu@hufs.ac.kr입니다. 추가적인 문의가 필요하시면 교수님께 직접 연락해 보시기 바랍니다. 혹시 더 도와드릴까요?


In [70]:
query1 = "기후변화융합학부도 외국어인증 필요해? 어떤 시험 봐야돼?"
nomal_query1 = normalize_query(query1)
result = chain.invoke({"input": nomal_query1}, config={"configurable": {"session_id": session_id}})

print(result["answer"])  # 챗봇 응답 출력

기후변화융합학부 소속 학생도 외국어인증이 필요합니다. 외국어인증 기준은 학부에 따라 다를 수 있으므로, 구체적인 시험 종류와 점수 기준은 해당 학부의 안내를 확인하시거나 학과사무실에 문의하시기 바랍니다. 일반적으로 FLEX, TOEIC, TOEFL 등의 시험이 인정됩니다.


In [71]:
query2 = "토익기준으로 몇 점 받아야돼?"
result = chain.invoke({"input": query2}, config={"configurable": {"session_id": session_id}})

print(result["answer"])  # 챗봇 응답 출력

기후변화융합학부의 경우, 토익 기준은 700점 이상입니다.


- 오답, 기후변화융합학부는 700점 이상요구됨
- 문제 상황: 검색된 문서에 정답 내용 포함 안됨, 기후변화융합학부를 자연대학의 일부로 추론할 가능성 높음
- 해결 방안: parent document Retriever, CoT 프롬프팅

> 1차수정
     > 파인콘 db 초기화 방식 수정 -> 정확한 응답 출력

In [17]:

texts = [doc.page_content for doc in doc_list]
bm25_retriever = BM25Retriever.from_texts(texts)
bm25_retriever.k = 20

pinecone_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, pinecone_retriever], weights=[0.3, 0.7])

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
reranker = CrossEncoderReranker(model=reranker_model, top_n=10)
compression_retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=ensemble_retriever)

# 1. 앙상블 리트리버 결과
ensemble_docs = ensemble_retriever.invoke(query)

# 2. rerank/compress된 문서
#reranker = CrossEncoderReranker(model=reranker_model, top_n=15)
#compressor = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=ensemble_retriever)
compression_retriever_docs = compression_retriever.invoke(query)

# 3. reordering 결과
reorder = LongContextReorder()
reordered_docs = reorder.transform_documents(compression_retriever_docs)


C:\Users\samsung-user\anaconda3\envs\boo_chat\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
for doc in ensemble_docs:
    print(doc.metadata)
    print(doc.page_content)


{'doc_type': '교수진 정보', '연구분야': '스포츠미디어, 스포츠방송저널리즘', '연구실': '정보없음', '이름': '이승구 교수', '이메일': 'natioaldocu@daum.net', '전화번호': '정보없음', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': '정보없음'}
이승구 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. 연구분야는 스포츠미디어, 스포츠방송저널리즘입니다. 이메일은 natioaldocu@daum.net입니다.
{'doc_type': '교수진 정보', '연구분야': '스포츠컴퓨터프로그래밍, 스포츠컴퓨터프로그래밍언어', '연구실': '공학관 416호', '이름': '김상철 교수', '이메일': 'kimsa@hufs.ac.kr', '전화번호': '031-330-4096', '직위': '교수', '학과': '글로벌스포츠산업학부', '학위': 'Ph.D Michigan State University, Engineering'}
김상철 교수 교수님은 글로벌스포츠산업학부 학과 소속입니다. 교수 직위를 맡고 있습니다. Ph.D Michigan State University, Engineering 학위를 가지고 있습니다. 연구분야는 스포츠컴퓨터프로그래밍, 스포츠컴퓨터프로그래밍언어입니다. 이메일은 kimsa@hufs.ac.kr입니다. 연구실은 공학관 416호입니다.
{'doc_type': '교수진 정보', '연구분야': '정보없음', '연구실': '정보없음', '이름': '이성준', '이메일': 's9114021@yahoo.com', '전화번호': '정보없음', '직위': '정보없음', '학과': '브라질학과', '학위': '정보없음'}
이성준 교수님은 브라질학과 학과 소속입니다. 이메일은 s9114021@yahoo.com입니다.
{'doc_type': '교수진 정보', '연구분야': 'E스포츠', '연구실': '정보없음', '이름': '서형석 교수', '이메일': 'esports@gma

In [53]:
for doc in compression_retriever_docs:
    print(doc.metadata)
    print(doc.page_content)


{}
## [heading1] ##
# 3. 이중전공 및 부전공(전공심화)

## [table] ##
| 대상 | 2007학번 이후 학생 |
| --- | --- |
| 내용 | ① 이중전공, ② 부전공, ③ 전공심화+부전공, ④ 전공심화(단일전공) 중 1개 과정을 선택하여 필수 이수 미선택시 자동으로 ④ 전공심화(단일전공) 이수하는 것으로 처리 ※ ② 부전공, ③ 전공심화+부전공의 졸업 학점이 다르니 반드시 확인 |
| 시작시기 | 2학년 (※ 편입생은 편입 두 번째 학기부터) |
| 배정시기 | 2학년 진급시 |
| ※ 필수 이수 제외자 | ❶ 편입생, ❷ 군위탁생, ❸ 순수외국인 특별전형 입학생(2015학번부터, 사범대학 학생 제외) ❹ 융합인재대학 소속 학생 |
| ※ 필수 이수 제외자 | ❶ ~ ❸ 학생은 ※ 이중전공, 부전공 또는 전공심화+부전공 이수 가능[단, 전공심화(단일전공)만은 불가] ※ 외국어인증 의무가 생길 수 있음[외국어인증(18페이지) 참조] |

## [footer] ##
- 70 -
{}
## [heading1] ##
# 2. 졸업시험 / 논문

## [list] ##
- 가. 1전공, 이중전공, 제2전공(2006학번 이전)은 반드시 졸업시험이나 논문을 통과하여야
- 졸업이 가능함
- 나. 매 학기 중반(4~5월, 10~11월)에 졸업시험 및 논문작성/심사 등을 진행하며, 학과별,
- 전공별 조건 및 일정은 각 학과사무실에 확인하여 기간 내 조치토록 함
- 다. 졸업시험/논문에 합격 후 졸업 시까지 그 자격이 유효하나, 합격 후 제적처리가 되는
- 경우에는 합격이 취소됨
- 라. 졸업시험FLEX와 외국어인증은 별도이므로 각각 요건을 확인하고 응시 / 처리하여야 함

## [paragraph] ##
3. 외국어인증(졸업인증) : 입학과 동시에 제출 가능

## [paragraph] ##
가. 외국어인증 : 2007학번 이후 모든 학생은 졸업 필수요건으로 외국어인증을 완료해야 함
(편입생, 군위탁생, 새터민은 인증 면제 / 2004~2

In [54]:
for doc in reordered_docs:
    print(doc.metadata)
    print(doc.page_content)

{}
## [heading1] ##
# 2. 졸업시험 / 논문

## [list] ##
- 가. 1전공, 이중전공, 제2전공(2006학번 이전)은 반드시 졸업시험이나 논문을 통과하여야
- 졸업이 가능함
- 나. 매 학기 중반(4~5월, 10~11월)에 졸업시험 및 논문작성/심사 등을 진행하며, 학과별,
- 전공별 조건 및 일정은 각 학과사무실에 확인하여 기간 내 조치토록 함
- 다. 졸업시험/논문에 합격 후 졸업 시까지 그 자격이 유효하나, 합격 후 제적처리가 되는
- 경우에는 합격이 취소됨
- 라. 졸업시험FLEX와 외국어인증은 별도이므로 각각 요건을 확인하고 응시 / 처리하여야 함

## [paragraph] ##
3. 외국어인증(졸업인증) : 입학과 동시에 제출 가능

## [paragraph] ##
가. 외국어인증 : 2007학번 이후 모든 학생은 졸업 필수요건으로 외국어인증을 완료해야 함
(편입생, 군위탁생, 새터민은 인증 면제 / 2004~2006학번은 아래 졸업인증 기준을 따름.
단, 편입생 및 군위탁생으로 면제대상이거나 2006학번 이전 학생이더라도 이중전공이
나 전공심화+부전공을 이수한 경우 외국어인증을 완료해야 함)
※ 외국어인증은 입학과 동시에 가능하며 졸업 전 미리 인증하는 것을 권고함
(단 제출시점 당시 증명서의 유효기간(기준 2년 이내 취득)이 남아 있어야 함)

## [paragraph] ##
※ 외국어인증 기준
{}
[공통] 외국어인증 신청서 제출 안내

[공통] 외국어인증 신청서 제출 안내

외국어인증 신청서 제출 안내

외국어인증 신청서 제출에 대해 아래와 같이 안내합니다 .

1. 외국어인증이란

서울캠퍼스 수강편람 바로가기

(수강편람목록- 수강편람 중 해당 페이지 참조)

글로벌캠퍼스 수강편람 바로가기

(수강편람목록- 수강편람 중 해당 페이지 참조 )

2. 외국어인증 신청서 제출 안내

제출서류

1. 외국어인증 신청서(첨부파일 참조)

2. 제출시점 기준 유효기간(2년) 이내의 인증 시험 성적 ( 또는 대체 과정 수료